# Spectral Line Data Cubes in Astronomy

In this notebook we will use spectral line data cubes in astronomy. They are a convenient way to store many spectra at points in the sky. Much like having a spectrum at every pixel in a CCD. Here we will keep it as much "pure python", and not use astronomical units and just work in "pixel" or "voxel" space.   

These typed of data are normally presented as a [FITS](https://en.wikipedia.org/wiki/FITS) file, with two sky coordinates (often Right Ascension and Declination, but sometimes Galactic Longitude and Galactic Latitude) and one spectral coordinate (either an observing frequency or wavelength, and when there is a known spectral line, you can reference using this line with a velocity using the doppler effect). For radio data, such as ALMA and the VLA, we often use GHz or MHz. For optical data we often use the Angstrom (the visible range is around 4000 - 8000 Angstrom, or 400 - 800 nm).

![Example Cube](cube_dims_and_cell.png "just an example cube")

# Predicting HI profiles 

This notebook was derived from a script **hi-observe.py**, which uses the command line. It exacts a spectrum from a FITS cube. The output figure can be compared to the strip chart you have obtained with the Greenbank 40ft.


In [ ]:
%matplotlib inline

import sys
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord


## Telescope Data

### LAB  (Leiden/Argentine/Bonn)survey

The **LAB.fits** file is a large HI survey. (Ref: [CDS](http://cdsarc.u-strasbg.fr/viz-bin/cat/VIII/76)). It is an all sky survey, taken with two nearly identical telescopes on the northern and southern hemisphere. It has a 0.5 degree (size of the sun/moon), so half the resolution as the Greenbank 40ft.

There is a copy in https://www.astro.umd.edu/~teuben/LAB.fits or https://www.astro.umd.edu/~teuben/LAB.zip

Make sure the FITS file is in the directory where this notebook is.

### BL: Bell Labs  survey

Alternatively, you can download **BL.fits**, the smaller and lower resolution Bell Labs Survey, which you can find referenced in [CDS](ftp://cdsarc.u-strasbg.fr/pub/cats/VIII/28). This is the one with a 2 degree beam, about the resolution of a rat, and twice the resolution of the Greenbank 40ft.


There is a copy in https://www.astro.umd.edu/~teuben/BL.fits or https://www.astro.umd.edu/~teuben/BL.zip


### DS9: viewing FITS files like images

There are many FITS viewers available, my favorite one is [ds9](https://sites.google.com/cfa.harvard.edu/saoimageds9).  In your free time, I encourage you to play with ds9 and one of the datasets.

Alternatives are:   [QFitsView](https://www.mpe.mpg.de/~ott/QFitsView/) and a more experimental [CARTA](https://cartavis.github.io/)


Now two cells with alternative download methods. You should be able to skip them.

In [ ]:
%%bash

if [ ! -e LAB.fits ]; then
   # there is also LAB.zip, but then you need more space, and run unzip LAB.zip
   wget https://www.astro.umd.edu/~teuben/LAB.fits
else
   echo You already have LAB.fits
fi

In [ ]:
%%bash

if [ ! -e BL.fits ]; then
   wget https://www.astro.umd.edu/~teuben/BL.fits
else
   echo You already have LAB.fits
fi

In [ ]:
fitsfile = 'LAB.fits'       #  (891, 361, 721) 460MB 
#fitsfile = 'BL.fits'       #  (145, 400, 800)  92MB

In [ ]:
!ls -l


In [ ]:
%%bash

ls -l

First two convenience functions that transform between the equatorial RA,DEC (or $\alpha,\delta$) and galatic GLON,GLAT ($l,b$) systems

In [ ]:
def radec_glonlat(rah,ram,ras,dec):
    """
    Simple conversion of 40ft RA(LST)/DEC to GLON/GLAT
    E.g. is LST = 12:10:30 and DEC=52 you would call
    this routine as   (glon,glat) = radec_glonlat(12,10,30,52)
    """
    r = rah*u.hour  + ram*u.minute + ras*u.second
    d = dec*u.deg
    c = SkyCoord(ra=r, dec=d, frame='icrs')
    g = c.galactic.to_string().split()
    return (float(g[0]),float(g[1]))


In [ ]:
def glonlat_radec(glon,glat):
    """
    Convert GLON/GLAT to RA/DEC
    """
    lon = glon*u.deg
    lat = glat*u.deg
    c = SkyCoord(l=lon, b=lat, frame='galactic')
    g = c.icrs.to_string().split()
    return (float(g[0]),float(g[1]))


It is always good to have a quick (regression) test to see if the function still works as when we wrote it.


In [ ]:
# testing the functions
(glon,glat) = radec_glonlat(21,0,0,42)
(ra,dec)    = glonlat_radec(glon,glat)
print("test1: radec_glonlat(20,58,15,42) should produce 83.886 -2.67928:   %g %g" % (glon,glat))
print("test2: glonlat_radec(glon,glat)   should produce 315 42:            %g %g" % (ra,dec))


Define some constants we might need (but actually not in the current version of this code)

In [ ]:
c        = 299792.458         # speed of light in km/s
restfreq = 1420.405751786     # HI restfreq in MHz



The wavelength and frequency of a wave is related to the transmission speed of the wave:
$$
     \lambda = {c \over \nu}
$$
where $\lambda$ and $\nu$ are the wavelength and frequency resp. We do need to put the numbers in the right units, though,
but here should be the expression:


Now we set the input parameters for the spectrum. The filename, and the position, but we allow 3 methods to enter the position.

For this project the easiest is a full sky HI survey, 
in the form of the **LAB.fits** file. It can be obtained from [ftp://cdsarc.u-strasbg.fr/pub/cats/VIII/76/lab.fit.gz](ftp://cdsarc.u-strasbg.fr/pub/cats/VIII/76/lab.fit.gz)
by uncompressing and renaming this appropriately.

In [ ]:
wavelength = (c*1000) / (restfreq*1000000)

print("Wavelength of HI = %g m" % wavelength)

In [ ]:
method = 2

if method == 1:                                           # give the GLON/GLAT
    xpos = 83.0
    ypos = -2.0
    have_pixel = False
elif method == 2:                                         # give the RA/DEC or Greenbank40ft the LST,DEC  
    rah = 20
    ram = 58
    ras = 15
    dec = 42
    (xpos,ypos)= radec_glonlat(rah,ram,ras,dec)
    have_pixel = False
else:                                                     # give the pixel coordinates
    xpos = 612
    ypos = 193
    have_pixel = True
    

In [ ]:
fitsfile = 'LAB.fits' 
fitsfile = 'BL.fits'

In [ ]:
# open the fits file (a FITS file has one or more Header-Data-Unit's, HDU's)
hdu = fits.open(fitsfile)

print("Opened FITS file %s with %d HDU's"  %  (fitsfile,len(hdu)))

In [ ]:
# get a reference to the primary header and data.
h = hdu[0].header
d = hdu[0].data          # sometimes you need data.squeeze() to get rid of redundant dimensions
print("Filename       :",fitsfile)
print("Shape of cube  :",d.shape)
if len(d.shape) != 3:                 # The data better be 3-dim numpy array now
    print("Your cube is not 3D")
    sys.exit(1)

# get the important coordinate conversion factors that scale between pixels and WCS

# axis 1 is GLON, in degrees
cdelt1 = h['CDELT1']
crval1 = h['CRVAL1']
crpix1 = h['CRPIX1']
ctype1 = h['CTYPE1']

# axis 2 is GLAT, in degrees
cdelt2 = h['CDELT2']
crval2 = h['CRVAL2']
crpix2 = h['CRPIX2']
ctype2 = h['CTYPE2']

# axis 3 is FELO, velocities in m/s (which we later convert to km/s)
cdelt3 = h['CDELT3']
crval3 = h['CRVAL3']
crpix3 = h['CRPIX3']
ctype3 = h['CTYPE3']

print("Cube axis names:  %s , %s , %s"  %  (ctype1,ctype2,ctype3))

Note that numpy arrays store the data in [VEL,GLAT,GLON] order, so we have 721 coordinates in RA in this example.

In [ ]:
if not have_pixel:
    # need to convert the xpos,ypos (in WCS) to pixel
    #    xpos_wcs = (xpos_pix - crpix + 1) * cdelt + crval
    #    xpos_pix = (xpos_wcs - crval)/cdelt + crpix - 1
    xposl = xpos
    yposb = ypos
    xpos = (xposl - crval1)/cdelt1 + crpix1 - 1 
    ypos = (yposb - crval2)/cdelt2 + crpix2 - 1
    print("Pixel: %g %g (converted from WCS %g %g)" % (xpos,ypos,xposl,yposb))
    xpos = int(xpos)
    ypos = int(ypos)

Now we grab get the spectrum using a numpy slice operation.
This will be the Y (intensity) coordinate in the spectrum plot below.


In [ ]:
flux     = d[:,ypos,xpos]

#   some helper arrays for the X (velocity) coordinate in the plot
nchan    = len(flux)       
zero     = np.zeros(nchan)
channeln = np.arange(nchan)
channelf = (channeln-crpix3+1)*cdelt3 + crval3   # WCS in m/s, notice channeln starts at 0
channelv = channelf / 1000.0                     # convert assumed m/s to km/s
print("MinMax in velocities:",channelv.min(), channelv.max())



In [ ]:
plt.plot(channelv,flux,'o-',markersize=2,label='HI-spectrum')
plt.plot(channelv,zero,                  label='baseline')
plt.xlabel("Doppler Velocity (km/s)")
plt.ylabel("Brightness")
plt.title("%s  @ %g %g" % (fitsfile,xpos,ypos))
plt.legend();

Finally just some statistics of the spectrum

In [ ]:
print("Mean and RMS of %d points: %g %g" % (len(flux),flux.mean(),flux.std()))